# Final project

This notebook together with the 'final_project' directory in this repository contains all the data and code used to create a final project on the course `Practical analysis of noisy and uneven time series` on `Advanced Data Analytics` master program on the University of Belgrade.
In the notebook, we will work with light curves with the main focus of applying prestatistical analyses using packages such as `numpy` and `pandas` and modelling neural processes with the implementation of `QNPy` a proprietary python package for modelling quasar time series using neural processes.

In [ ]:
__author__ = 'Damir Bogdan <damirbogdan39@gmail.com>'
__version__ = '20230411'
__keywords__ = ['lightcurve', 'quasars', 'clustering', 'nerual processes']

In [ ]:
# Importing dependencies

import os
from glob import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Utilities
from utils import count_outliers


In [ ]:
# QNPy dependencies

import QNPy #Importing the package

# Preprocessing
from QNPy import Preprocess as pr #Importing Preprocess module from the package
from QNPy.Preprocess import transform #importing the funcion transform for transformation the data
from QNPy.Preprocess import * #importing all external packages from Preprocess

# Splitting and training
from QNPy import SPLITTING_AND_TRAINING as st
from QNPy.SPLITTING_AND_TRAINING import *

# Prediction 
from QNPy import PREDICTION as pred #Importing PREDICTION module from the package
from QNPy.PREDICTION import * #Importing all packages from PREDICTION module
from QNPy.PREDICTION import plot_function #The functions plot_function must be imported separately

In [ ]:
# Defining data path

data_path = 'kriveu/'

In the next cell we will import all individual light curves as a dataframe. This will allow us to inspect every individual light curve using common padnas functions but also visualize them.

In [ ]:
# Importing the light curves as dataframes

# Get a list of all files in the folder
all_files = os.listdir(data_path)

all_dataframes = []

# Read each file into a DataFrame and store it in the list
for file_name in all_files:
    try:
        # Construct the full file path
        file_path = os.path.join(data_path, file_name)
        
        # Read the DataFrame and create a variable with the file name
        df = pd.read_csv(file_path)
        globals()['lc_' + os.path.splitext(file_name)[0]] = df
        
        # Add the DataFrame to the list
        all_dataframes.append(df)
    except Exception as e:
        print(f"Error reading {file_name}: {e}")


We also create a combined dataframe called `combined_df` so we can inspect them all as one time series.

In [ ]:
# Create a combined dataframe

# Create an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Iterate through each file and read it into a DataFrame
for file in all_files:
    # Construct the full file path
    file_path = os.path.join(data_path, file)
    
    # Read the file into a DataFrame
    df = pd.read_csv(file_path)  # Adjust the read function based on your file format
    
    # Add a new column with the file name
    df['light_curve'] = file
    
    # Concatenate the current DataFrame with the combined DataFrame
    combined_df = pd.concat([combined_df, df], ignore_index=True)

## EDA

### Individual light curves

In [ ]:
# Loop to print the shape and name of each DataFrame
for file_name, df in zip(all_files, all_dataframes):
    df_name = 'lc_' + os.path.splitext(file_name)[0]
    print(f"{df_name}: {df.shape}")

We can see that our dataframes are of different shapes.
Their first dimension differs which means they are not equal in the number of rows which represent the number of timestamps. The smallest light curve consists out of 107 records, and the largest one consist out of 151 records.


In [ ]:
# Visualizing lightcurves

for i, df in enumerate(all_dataframes):
    fig = go.Figure()

    trace = go.Scatter(x=df['mjd'], y=df['mag'], mode='markers', name='mag', marker=dict(size=4))

    error_bars = go.Scatter(
        x=df['mjd'],
        y=df['mag'],
        error_y=dict(
            type='data',
            array=df['magerr'],
            visible=True
        ),
        mode='markers',
        marker=dict(size=4),
        name='mag with error bars'
    )

    fig.add_trace(trace)
    fig.add_trace(error_bars)

    fig.update_xaxes(title_text='MJD (Modified Julian Date)')
    fig.update_yaxes(title_text='Magnitude')

    fig.update_layout(title_text=f"Time Series with Error Bars - Plot {i + 1}", showlegend=True)
    fig.show()


In [ ]:
# Visualizing box and whisker plots and histograms for all individual light curves

for i, df in enumerate(all_dataframes):
    fig = px.histogram(df, x='mag', marginal='box', nbins=30, title=f"KDE Plot for 'mag' - Plot {i + 1}")

    fig.update_xaxes(title_text='Magnitude')
    fig.update_yaxes(title_text='Density')

    fig.show()

In [ ]:
# Get descriptive statistics of each light curve

combined_df.groupby('light_curve')[['mag','magerr']].describe()

In [ ]:
# Ge the number of outliers in each light curve

combined_df.groupby('light_curve')[['mag','magerr']].apply(count_outliers)

### All light curves together

In [ ]:
combined_df.info()

In [ ]:
# Visualize all light curves on one graph

fig = px.scatter()

colors = px.colors.qualitative.Set1 + px.colors.qualitative.Pastel1 # Combine two color palletes because Set1 isn't enough

for i, df in enumerate(all_dataframes):
    trace = go.Scatter(
        x=df['mjd'],
        y=df['mag'],
        mode='markers',
        name=f'Plot {i + 1}',
        marker=dict(size=4, color=colors[i % len(colors)])
    )

    error_bars = go.Scatter(
        x=df['mjd'],
        y=df['mag'],
        error_y=dict(
            type='data',
            array=df['magerr'],
            visible=True
        ),
        mode='markers',
        marker=dict(size=4, color=colors[i % len(colors)]),
        showlegend=False
    )

    fig.add_trace(trace)
    fig.add_trace(error_bars)

fig.update_xaxes(title_text='MJD (Modified Julian Date)')
fig.update_yaxes(title_text='Magnitude')

fig.update_layout(title_text="Time Series with Error Bars - All Plots", showlegend=True)
fig.show()


## QNPy implementation

### Preprocessing

In [ ]:
path = "./light_curves"
files = glob.glob(path + "/*.csv")
df_list = (pd.read_csv(file) for file in files)
data = pd.concat(df_list, ignore_index=True)

In [ ]:
# Padding the light curves

padding = pr.backward_pad_curves('./light_curves', './padded_lc', desired_observations=100)

In [ ]:
# Path to padded data 
PADDED_PATH = "./padded_lc"

# Path to save preprocesses data
PREPROC_PATH = "./preproc"

In [ ]:
padded_files = os.listdir(PADDED_PATH)
padded_files

In [ ]:
number_of_points, trcoeff = pr.transform_and_save(padded_files, PADDED_PATH, PREPROC_PATH, transform)

### Splitting the data

In [ ]:
preproc_files = os.listdir(PREPROC_PATH) #listing the transformed data

In [ ]:
# Defining the paths to train, val and test directories

TRAIN_DIR = "./dataset/train/"
VAL_DIR = "./dataset/val/"
TEST_DIR = "./dataset/test/"

In [ ]:
# Splitting the data

# st.split_data(preproc_files, PREPROC_PATH, TRAIN_DIR, TEST_DIR, VAL_DIR)

### Training the model

In [ ]:
# Defining the paths to the directories

TRAIN_DIR_PATH = "./dataset/train"
VAL_DIR_PATH = "./dataset/val"
TEST_DIR_PATH = "./dataset/test"

# Defining the path to the model
MODEL_PATH = "./output/cnp_model.pth"

In [ ]:
# Defining batch size 

BATCH_SIZE = 32

In [ ]:
# Creating train and validation loaders

train_loader, val_loader = st.get_data_loaders(TRAIN_DIR_PATH, VAL_DIR_PATH, BATCH_SIZE)

In [ ]:
# Defining device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Creating the model and optimizer

model, optimizer, criterion, mse_metric, mae_metric = st.create_model_and_optimizer(device)

In [ ]:
# Training the model

history_loss_train, history_loss_val, \
history_mse_train, history_mse_val, \
history_mae_train, history_mae_val, \
epoch_counter_train_loss, epoch_counter_train_mse, epoch_counter_train_mae, \
epoch_counter_val_loss, epoch_counter_val_mse, epoch_counter_val_mae = \
st.train_model(model,
               train_loader, val_loader,
               criterion, optimizer,
               1, 6000, 3000,
               mse_metric, mae_metric,
               device)

In [ ]:
# Define the file names
file_names = ["history_loss_train.csv", "history_loss_val.csv",
              "history_mse_train.csv", "history_mse_val.csv",
              "history_mae_train.csv", "history_mae_val.csv",
              "epoch_counter_train_loss.csv", "epoch_counter_train_mse.csv", "epoch_counter_train_mae.csv",
              "epoch_counter_val_loss.csv", "epoch_counter_val_mse.csv", "epoch_counter_val_mae.csv"]

# Define the lists
lists = [history_loss_train, history_loss_val,
         history_mse_train, history_mse_val,
         history_mae_train, history_mae_val,
         epoch_counter_train_loss, epoch_counter_train_mse, epoch_counter_train_mae,
         epoch_counter_val_loss, epoch_counter_val_mse, epoch_counter_val_mae]

#Running the function for saving all lists with histories
save_list = st.save_lists_to_csv(file_names, lists)

ADD COMMENT FOR COUNTERS

In [ ]:
# Replace with the path to your history_loss_train CSV file
history_loss_train_file = './history_loss_train.csv'  
# Replace with the path to your history_loss_val CSV file
history_loss_val_file = './history_loss_val.csv'  
# Replace with the path to your epoch_counter_train_loss CSV file
epoch_counter_train_loss_file = './epoch_counter_train_loss.csv'  

In [ ]:
directory_path = "./"

csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

def create_training_df(csv_paths):
    
    dataframes=[]

    for path in csv_paths:
        df = pd.read_csv(path, header=None, sep=',').T
        df.rename({0:path[:-4]}, axis=1, inplace=True)
        dataframes.append(df)
    
    training_df = pd.concat(dataframes, axis=1)

    return training_df


training_df = create_training_df(csv_files)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

# Assuming 'epoch_counter_train_loss', 'history_loss_train', and 'history_loss_val' are columns in training_df

fig = go.Figure()

fig.add_trace(go.Scatter(x=training_df['epoch_counter_train_loss'], y=training_df['history_loss_train'], mode='lines', name='Train LOSS'))
fig.add_trace(go.Scatter(x=training_df['epoch_counter_train_loss'], y=training_df['history_loss_val'], mode='lines', name='Validation LOSS'))

fig.update_layout(
    title="LogProbLOSS",
    xaxis_title="Epoch",
    yaxis_title="LOSS",
    legend=dict(x=0, y=1, traceorder='normal', orientation='h'),
)

fig.show()


In [ ]:
#plotting the Logprobloss after training
logprobloss = st.plot_loss(history_loss_train_file, history_loss_val_file, epoch_counter_train_loss_file)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.lineplot(x='epoch_counter_train_loss', y='history_loss_train', data=training_df, label='Train LOSS')
sns.lineplot(x='epoch_counter_train_loss', y='history_loss_val', data=training_df, label='Validation LOSS')
plt.title("LosProbLOSS")
plt.xlabel("Epoch")
plt.ylabel("LOSS")
plt.legend()
plt.show()


In [ ]:
 # Replace with the path to your history_mse_train CSV file
history_mse_train_file = './history_mse_train.csv'
# Replace with the path to your history_mse_val CSV file
history_mse_val_file = './history_mse_val.csv'  
# Replace with the path to your epoch_counter_train_mse CSV file
epoch_counter_train_mse_file = './epoch_counter_train_mse.csv'  

In [ ]:
#plotting the MSE metric after training
msemetric=st.plot_mse(history_mse_train_file, history_mse_val_file, epoch_counter_train_mse_file)

In [ ]:
# Replace with the path to your history_mae_train CSV file
history_mae_train_file = './history_mae_train.csv'
# Replace with the path to your history_mae_val CSV file
history_mae_val_file = './history_mae_val.csv'  
# Replace with the path to your epoch_counter_train_mae CSV file
epoch_counter_train_mae_file = './epoch_counter_train_mae.csv'  

In [ ]:
#plotting the MAE metric after training
maemetric=st.plot_mae(history_mae_train_file, history_mae_val_file, epoch_counter_train_mae_file)

In [ ]:
# Saving the trained model

save = st.save_model(model, MODEL_PATH)

In [ ]:
OUTPUT_PATH = './output/predictions'

In [ ]:
# This function call is optional, it is meant to be executed when output and it's child directiores are not empty.

pred.prepare_output_dir(OUTPUT_PATH)

In [ ]:
# Loading the model load_trained_model function

model = pred.load_trained_model(MODEL_PATH, device)

ADD THAT YOU HAD TO TROUBLESHOOT WHY PR WOULD NOT WORK. IT'S BECASE THE PREPROCESSING IS CALLED PR AND ALSO PREDICTION IS CALLED PR

In [ ]:
criterion, mse_metric = pred.get_criteria()

In [ ]:
if __name__ == "__main__":
    folder_path = "./dataset/test"

    pred.remove_padded_values_and_filter(folder_path)

In [ ]:
if __name__ == "__main__":
    folder_path = "./dataset/val"

    pred.remove_padded_values_and_filter(folder_path)


In [ ]:
if __name__ == "__main__":
    folder_path = "./dataset/val"

    pred.remove_padded_values_and_filter(folder_path)

In [ ]:
test_loader = pred.load_test_data(TEST_DIR_PATH)

In [ ]:
train_loader = pred.load_train_data(TRAIN_DIR_PATH)

In [ ]:
val_loader = pred.load_val_data(VAL_DIR_PATH)

In [ ]:
test_metrics = pred.plot_light_curves_from_test_set(model, test_loader, criterion, mse_metric, plot_function, device)

In [ ]:
save_test_metric = pred.save_test_metrics('./output/predictions/', test_metrics)

In [ ]:
#prediction and plotting the train data
train_metrics=pred.plot_light_curves_from_train_set(train_loader, model, criterion, mse_metric, plot_function, device)

In [ ]:
save_train_metric=pred.save_train_metrics('./output/predictions/', train_metrics)#saving the train metrics 

In [ ]:
val_metrics=pred.plot_light_curves_from_val_set(model, val_loader, criterion, mse_metric, plot_function, device)

In [ ]:
save_val_metrics=pred.save_val_metrics('./output/predictions/', val_metrics)